In [ ]:
%cd ../../src


In [5]:
from dataload import dmg777k
data = dmg777k()

loaded data dmg777k (70.45s).
pruned (12.38s).


In [14]:
from utils import get_relevant_relations, ALL_LITERALS
import torch
rr = get_relevant_relations(data, ALL_LITERALS)
for r in rr:
    df = data.triples[data.triples[:,1]==r]
    new_df = df.clone().detach()

    new_df[:, 1] = data.num_relations
    new_df[:, 2] = data.num_entities

    data.i2r.append(f"https://master-thesis.com/relations#one-relation-{r}")
    data.r2i[f'https://master-thesis.com/relations#one-relation-{r}'] = data.num_relations

    data.i2e.append(
                (f'https://master-thesis.com/entitys#one-literal-{r}', 'preprocessed'))
    data.e2i[(
                f'https://master-thesis.com/entitys#one-literal-{r}', 'preprocessed')] = data.num_entities

    data.triples = torch.cat((data.triples, new_df), 0)
    data.num_relations += 1
    data.num_entities += 1


In [8]:
data.triples[data.triples[:,1]]

tensor([[130685,     28,  54795],
        [130685,     31, 201822],
        [130690,     28,  58948],
        ...,
        [271591,     31, 211084],
        [271593,     31, 211084],
        [271598,     31, 211084]], dtype=torch.int32)

In [ ]:
def bin_numbers(data: Data, num_bins=3):
    relevent_relations = get_relevant_relations(
        data, relevant_types=RDF_NUMBER_TYPES)

    i = 0
    cumsum = 0
    bins = np.arange(num_bins)

    for r in relevent_relations:
        current_bins = 0
        df_sub = data.triples[data.triples[:, 1] == r]
        # bins = np.digitize(sub_list,np.histogram(sub_list,num_bins,density = True)[1])
        sub_list = np.array([data.i2e[x][0]
                            for x in df_sub[:, 2]], dtype=np.float32)
        # mean = torch.round(torch.mean(df_sub[:,2],dtype = torch.float64)).to(torch.long)

        new = torch.tensor(df_sub[:, 0])

        new_df = torch.ones((new.size()[0], 3), dtype=torch.long)

        new_df[:, 0] = torch.tensor(df_sub[:, 0])  # values at place 0 (iri)
        new_df[:, 1] = data.num_relations + i
        binned_list = torch.from_numpy(np.digitize(
            sub_list, np.histogram(sub_list, num_bins)[1]))
        bi = 0
        while len(np.unique(binned_list)) < num_bins - bi:

            # print(f'{len(np.unique(binned_list))} vs {num_bins-bi}')
            bi += 1
            binned_list = torch.from_numpy(np.digitize(
                sub_list, np.histogram(sub_list, num_bins - bi)[1]))

        current_bins = len(np.unique(binned_list))
        new_df[:, 2] = binned_list
        new_df[:, 2] += (data.num_entities + ((i - 1) * num_bins)) - 1

        data.i2r.append(
            f"https://master-thesis.com/relations#sanity-check-{i}")
        data.r2i[f'https://master-thesis.com/relations#sanity-check-{i}'] = data.num_relations + i

        for e in range(current_bins):
            data.i2e.append(
                (f'https://master-thesis.com/entitys#sanity-check-target-{i}-{e}', 'preprocessed'))
            data.e2i[(
                f'https://master-thesis.com/entitys#sanity-check-target-{i}-{e}', 'preprocessed')] = data.num_entities + cumsum + e

        # i += 1
        # cumsum += current_bins

        # print(new_df)
        # break
        data.triples = torch.cat((data.triples, new_df), 0)
    data.num_relations += 1
    data.num_entities += num_bins

    return data